#Deep Learning a Vizuális Informatikában
##2. Házi Feladat

###1. Rész

Valósíts meg egy paraméterezhető konvolúciós neurális hálózatot, amely osztályozásra képes. A hálózat paraméterei a következők:


*   nC: Az osztályok száma
*   nFeat: Az első réteg kimeneti csatornaszáma. Az ezt követő rétegek be- és kimeneti csatornaszáma egyezzen ezzel meg, majd minden leskálázó (strided konvolúciós) réteg duplázza ezt meg.
*   nLevels: A háló szintjeinek száma. Egy szintnek az azonos térbeli kiterjedésű tenzorokon operáló rétegeket nevezzük (2 leskálázás közt). (Tipp: használj adaptív poolingot az osztályozó réteg előtt, hogy a változó szint szám ne okozzon problémát.)
*   layersPerLevel: Az egy szinten található konvolúciós rétegek száma.
*   kernelSize: A konvolúciós rétegek mérete.
*   nLinType: Kategorikus változó, amellyel a nemlinearitás típusát állíthatja (hogy hány és milyen függvények közül lehet választani önre van bízva)
*   bNorm: Bináris változó, amellyel állítható, hogy a konvolúciós rétegekbe teszünk-e BatchNormot.
*   residual: Bináris változó, True érték esetén minden szinten valósíts meg egy reziduális kapcsolatot a szint bemenete és a szint végén megjelenő leskálázó réteg bemenete közt.

Tipp: Érdemes ehhez írni először két külön modult, ami egy réteget (batchnormmal, dropouttal és nemlinearitással) valósít meg és egyet, ami meg ezekből egy szintet legózik össze. Ezekből aránylag könnyen összelegózható a háló.

Tipp 2: A PyTorchnak van nn.Sequential osztálya. Ez egy lista, amiben rétegek vannak, de ha sima lista változóba tesztek egyszerre több nn.Module-t, annak az optimizer nem fogja megkapni. Ez azért van, mert amikor egy nn.Module-tól leszármazó osztálytól elkéritek a .parameters()-t, akkor az végignézi az objektum összes tagváltozót, hogy van-e neki .parameters() függvénye (és ezt szépen rekurzívan végigcsinálja). A Listának pedig nincs, hiába vannak benne olyan elemek, amiknek van. Ezen felül az nn.Sequential forward függvénye is felül van csapva, és szépen sorban meghívja a belül lévő rétegeket.



In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernelSize, nLinType, bNorm, is_strided=False):
        super(CustomConvLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernelSize, stride=(2 if is_strided else 1),
                              padding=kernelSize//2 + (1 if is_strided else 0))
        if nLinType == 1:
            self.nonlin = nn.ReLU()
        elif nLinType == 2:
            self.nonlin = nn.LeakyReLU()
        elif nLinType == 3:
            self.nonlin = nn.ELU()
        else:
            raise ValueError("Invalid nLinType value")

        self.bNorm = bNorm
        if bNorm:
            self.bn = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout()   # During training, randomly zeroes some of the elements of the input tensor with probability p.

    def forward(self, x):
        x = self.conv(x)
        x = self.nonlin(x)
        if self.bNorm:
            x = self.bn(x)
        return self.dropout(x)

class CustomLevel(nn.Module):
    def __init__(self, in_channels, out_channels, layersPerLevel, kernelSize, nLinType, bNorm, residual):
        super(CustomLevel, self).__init__()
        layers = []
        for idx in range(layersPerLevel):
            layers.append(CustomConvLayer(in_channels, in_channels, kernelSize, nLinType, bNorm))
        self.layers = nn.Sequential(*layers)
        self.strided_conv = CustomConvLayer(in_channels, out_channels, kernelSize, nLinType, bNorm, True)
        self.residual = residual

    def forward(self, x):
        out = self.layers(x)
        if self.residual:
            out += x
        out = self.strided_conv(out)
        return out

class CustomCNN(nn.Module):
    def __init__(self, nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm, residual):
        super(CustomCNN, self).__init__()
        self.first_level = CustomLevel(3, nFeat, layersPerLevel, kernelSize, nLinType, bNorm, residual)
        self.levels = nn.ModuleList()
        channels = nFeat
        for idx in range(1, nLevels):
            self.levels.append(CustomLevel(channels, channels * 2, layersPerLevel, kernelSize, nLinType, bNorm, residual))
            channels *= 2  # Double channels after each level
        self.final_pool = nn.AdaptiveAvgPool2d(1)  # Adaptive pooling for final classification
        self.classifier = nn.Linear(channels, nC)

    def forward(self, x):
        x = self.first_level(x)
        for level in self.levels:
            x = level(x)
        x = self.final_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


###2. Rész

Valósíts meg egy paraméterezhető neurális háló tanító függvényt.

A függvény bemenetként megkapja a fenti neurális háló megkostruálásához szükséges paramétereket, az összes random seedet 42 értékre állítja, majd végrehajtja a neurális háló tanítását a kiadott adatbázison.

Használj Adam optimizert és Cosine Annealing tanulási ráta ütemezőt.

A tanítás során minden epoch után validálj, és jegyezd fel a legjobb validációs pontosságot, és a tanítás végén ezt add vissza.

A függvénynek további bemeneti paraméterei:

*   bSize: A bacth méret
*   lr: a tanulási ráta
*   lr_ratio: a tanulási ráta ütemező eta_min paramétere és a kezdeti tanulási ráta hányadosa
*   numEpoch: az epochok száma
*   decay: a weight_decay paraméter értéke



### Progress Bar

In [2]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_neural_network(nC, nFeat, nLevels, layersPerLevel, kernelSize,
                         nLinType, bNorm, residual, bSize, lr,
                         lr_ratio, numEpoch, decay, train_loader, val_loader):
    # Set random seeds
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Construct the neural network
    model = CustomCNN(nC, nFeat, nLevels, layersPerLevel, kernelSize, nLinType, bNorm, residual)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=numEpoch, eta_min=lr * lr_ratio)

    # Training loop
    best_val_accuracy = 0.0
    for epoch in range(numEpoch):
        #progress(epoch, numEpoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{numEpoch}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / val_total
        print(f"Validation accuracy after epoch {epoch+1}: {val_accuracy}")

        # Update best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy

        # Adjust learning rate
        scheduler.step()

    return best_val_accuracy


### Adatbázis letöltése

In [4]:
!wget http://deeplearning.iit.bme.hu/Public/Small.zip --no-check-certificate
!unzip -qq Small.zip
!rm Small.zip

--2024-05-08 06:29:34--  http://deeplearning.iit.bme.hu/Public/Small.zip
Resolving deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)... 152.66.243.112
Connecting to deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)|152.66.243.112|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://deeplearning.iit.bme.hu/Public/Small.zip [following]
--2024-05-08 06:29:37--  https://deeplearning.iit.bme.hu/Public/Small.zip
Connecting to deeplearning.iit.bme.hu (deeplearning.iit.bme.hu)|152.66.243.112|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 7048374 (6.7M) [application/zip]
Saving to: ‘Small.zip’

Small.zip           100%[===================>]   6.72M  5.72MB/s    in 1.2s    

2024-05-08 06:29:38 (5.72 MB/s) - ‘Small.zip’ saved [7048374/7048374]



In [56]:
import torchvision
import torchvision.transforms as transforms

train_dataset = torchvision.datasets.ImageFolder("Small/Classification/train/", transforms.ToTensor())
val_dataset = torchvision.datasets.ImageFolder("Small/Classification/val/", transforms.ToTensor())

nC = 5
nFeat = 8
nLevels = 4
layersPerLevel = 3
kernelSize = 3
nLinType = 1  # ReLU activation
bNorm = True
residual = True
bSize = 32
lr = 0.001
lr_ratio = 0.5
numEpoch = 20
decay = 0.001

# Example of training and validation data loaders (replace these with your own DataLoader instances)
train_loader = DataLoader(train_dataset, batch_size=bSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=bSize)

# Training the neural network
best_val_accuracy = train_neural_network(nC, nFeat, nLevels, layersPerLevel,
                                         kernelSize, nLinType, bNorm, residual,
                                         bSize, lr, lr_ratio, numEpoch, decay,
                                         train_loader, val_loader)

print(f"Best validation accuracy: {best_val_accuracy}")

Epoch 1/30: 100%|██████████| 79/79 [00:05<00:00, 15.02it/s]


Validation accuracy after epoch 1: 0.2


Epoch 2/30: 100%|██████████| 79/79 [00:05<00:00, 14.42it/s]


Validation accuracy after epoch 2: 0.2


Epoch 3/30: 100%|██████████| 79/79 [00:05<00:00, 13.68it/s]


...

Validation accuracy after epoch 28: 0.624


Epoch 29/30: 100%|██████████| 79/79 [00:05<00:00, 13.70it/s]


Validation accuracy after epoch 29: 0.5


Epoch 30/30: 100%|██████████| 79/79 [00:04<00:00, 15.81it/s]


Validation accuracy after epoch 30: 0.466
Best validation accuracy: 0.624


###3. Rész

Valósíts meg hiperparaméter optimalizálást a Bayesian Optimization python könyvtár felhasználásával. A könyvtár itt érhető el: https://github.com/fmfn/BayesianOptimization

A megoldás során a következőkre ügyelj:


1.   A kezdeti random lépések száma legyen kb egyenlő a szabad paraméterek számával (8). A bináris és kategorikus változókat nem érdemes optimalizálni külön, és az epochszámot is érdemes fixen megválasztani.
2.   Az teljes lépésszám legyen ennek 4-szerese (32)
3.   Mivel a Bayesian Optimization függvény a bináris/diszkrét/integer paramétereket nem támogatja, ezért a folytonos értékek megfelelő konverziója az előző feladatrészben megvalósított függvény feladata. Itt külön figyeljetek arra, hogy vannak olyan változók (pl a szűrők alap száma, vagy a szűrő mérete), amiknek csak bizonyos értékek értelmesek.
4.   Az epochok közben menő progress barokat meg kiíratásokat érdemes eltüntetni, a Bayesian opt majd fog írogatni
5.   Referenciaként, az adatbázison olyan 94-5% pontosság az elfogadható és 96%+ számít jónak.
6.   Ha megvan a végső legjobb paraméter, akkor érdemes azzal egy kicsit hosszabb tanítást lefuttatni.

Hogy a paramétereket milyen tartományban akarjátok optimalizálni, azt nektek kellene kitalálni. De a szintek számát és a szűrőméretet érelmetlen 5 fölé, a rétegek számát meg 3 fölé vinni. Az epochszámmal se menjetek 20 fölé, mert így is kb 1 óráig tart egy optimalizálás.



In [57]:
!pip install bayesian-optimization

In [66]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import UtilityFunction


def train_wrapper(nFeat, nLevels, layersPerLevel, kernelSize, lr, lr_ratio, decay):
  train_dataset = torchvision.datasets.ImageFolder("Small/Classification/train/", transforms.ToTensor())
  val_dataset = torchvision.datasets.ImageFolder("Small/Classification/val/", transforms.ToTensor())

  nFeat = int(nFeat)
  nLevels = int(nLevels)
  layersPerLevel = int(layersPerLevel)
  kernelSize = int(kernelSize) + 1 - int(kernelSize) % 2 # Make the size odd
  nC = 5
  nLinType = 1  # ReLU activation
  bNorm = True
  residual = True
  bSize = 32
  numEpoch = 15

  # Example of training and validation data loaders (replace these with your own DataLoader instances)
  train_loader = DataLoader(train_dataset, batch_size=bSize, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=bSize)

  return train_neural_network(nC, nFeat, nLevels, layersPerLevel,
                                         kernelSize, nLinType, bNorm, residual,
                                         bSize, lr, lr_ratio, numEpoch, decay,
                                         train_loader, val_loader)

pBounds = {
            'nFeat': (3, 16),
            'nLevels': (3, 5),
            'layersPerLevel': (2, 5),
            'kernelSize':(3, 5),
            'lr':(0.001, 0.1),
            'lr_ratio': (0.1, 0.9),
            'decay': (0.001, 0.1)
          }

hiper_optimizer = BayesianOptimization(
  f=train_wrapper,
  pbounds=pBounds,
  random_state=42
)

logger = JSONLogger(path="./hiper_opt.log")
hiper_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

#utility = UtilityFunction(kind="ucb", kappa=2.5, xi=0.0)
#hiper_optimizer.set_gp_params(normalize_y=True)

hiper_optimizer.maximize(
    init_points=8,
    n_iter=24,
    #acq=utility
)

print(f"The best combination of hiper-parameters after optimization:\n {hiper_optimizer.max}")

Epoch 1/15: 100%|██████████| 79/79 [00:08<00:00,  9.13it/s]


Validation accuracy after epoch 1: 0.2


Epoch 2/15: 100%|██████████| 79/79 [00:05<00:00, 13.20it/s]


Validation accuracy after epoch 2: 0.28


Epoch 3/15: 100%|██████████| 79/79 [00:07<00:00, 11.18it/s]


...

Validation accuracy after epoch 13: 0.703


Epoch 14/15: 100%|██████████| 79/79 [00:08<00:00,  9.31it/s]


Validation accuracy after epoch 14: 0.866


Epoch 15/15: 100%|██████████| 79/79 [00:08<00:00,  9.20it/s]


Validation accuracy after epoch 15: 0.802
The best combination of hiper-parameters after optimization:
 {'target': 0.911, 'params': {'decay': 0.058032476611574485, 'kernelSize': 4.052478729438609, 'layersPerLevel': 3.4423766758699026, 'lr': 0.002132814476577171, 'lr_ratio': 0.4875402239925086, 'nFeat': 9.549161872864335, 'nLevels': 3.0593686299768645}}


In [72]:
# Hosszabb tanitas az optimalizalt parameterekkel:

train_dataset = torchvision.datasets.ImageFolder("Small/Classification/train/", transforms.ToTensor())
val_dataset = torchvision.datasets.ImageFolder("Small/Classification/val/", transforms.ToTensor())

nFeat = int(hiper_optimizer.max['params']['nFeat'])
nLevels = int(hiper_optimizer.max['params']['nLevels'])
layersPerLevel = int(hiper_optimizer.max['params']['layersPerLevel'])
kernelSize = int(hiper_optimizer.max['params']['kernelSize']) + 1 - int(hiper_optimizer.max['params']['kernelSize']) % 2 # Make the size odd
lr = hiper_optimizer.max['params']['lr']
lr_ratio = hiper_optimizer.max['params']['lr_ratio']
decay = hiper_optimizer.max['params']['decay']
nC = 5
nLinType = 1  # ReLU activation
bNorm = True
residual = True
bSize = 32
numEpoch = 30
# Example of training and validation data loaders (replace these with your own DataLoader instances)
train_loader = DataLoader(train_dataset, batch_size=bSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=bSize)

best_val_accuracy_after_opt = train_neural_network(nC, nFeat, nLevels, layersPerLevel,
                                        kernelSize, nLinType, bNorm, residual,
                                        bSize, lr, lr_ratio, numEpoch, decay,
                                        train_loader, val_loader)

print(f"Best accuracy after hiper-param-optimization is {best_val_accuracy_after_opt}")

Epoch 1/30: 100%|██████████| 79/79 [00:09<00:00,  8.53it/s]


Validation accuracy after epoch 1: 0.385


Epoch 2/30: 100%|██████████| 79/79 [00:08<00:00,  9.48it/s]


Validation accuracy after epoch 2: 0.221


Epoch 3/30: 100%|██████████| 79/79 [00:08<00:00,  9.54it/s]


...

Validation accuracy after epoch 28: 0.829


Epoch 29/30: 100%|██████████| 79/79 [00:08<00:00,  9.26it/s]


Validation accuracy after epoch 29: 0.901


Epoch 30/30: 100%|██████████| 79/79 [00:07<00:00, 10.69it/s]


Validation accuracy after epoch 30: 0.896
Best accuracy after hiper-param-optimization is 0.911
